# DICE Model (at home)
**Exercise Session Resource Economics (Spring Term 2025)** \
Raul Hochuli (raul.hochuli@unibas.ch)

This is the second part of teaching the build up of the DICE model. **Please try to complete the Tasks listed in the notebook before class on Monday, 14th April 2025**. I ask you once again to email my any solutions you have until Sunday evening before the class, so I can try to adjust my teaching accordingly. 

## Preamble


The following code has been replicated and refactored for this course based on the inital material from [Hazem Krichene](https://github.com/hazem2410) in his public Github repository [PYDICE](https://github.com/hazem2410/PyDICE), which builds on the [DICE (Dynamic Integrated Climate Economy) Model of William D. Nordhaus](https://williamnordhaus.com/dicerice-models). 

### Naming Convention Table

| **Lecture**     | **Exercise Session**              | **Default Values**     | **Description**                                                                 |
|-----------------|-----------------------------------|------------------------|---------------------------------------------------------------------------------|
| $T$             | `T_end`                           | 100                    | Number of periods                                                               |
|                 | `tstep`                           | 5                      | Time step size                                                                  |
|                 | `scale1`                          | 0.0302455265681763     | Multiplicative scaling coefficient                                              |
|                 | `scale2`                          | -10993.704             | Additive scaling coefficient                                                    |
| $A$             | `TFPr`                            | exogenous              | Total factor productivity                                                       |
| $L$             | `Labr`                            | exogenous              | World population (millions, 2015: 7403; 2050: 11500)                            |
| $\sigma$        | `Dcrb`                            | exogenous              | Decarbonization rate (GTC / trillion 2010 USD)                                  |
| $\theta_1$      | `cost1_Abat`                      | exogenous              | Abatement cost multiplicative term                                              |
| $E_{Land}$      | `CEms_land`                       | exogenous              | Carbon emissions through land use (deforestation), GtC                          |
| $S$             | `Savi`                            | decision variable      | Savings rate                                                                    |
| $\mu$           | `ECtr`                            | decision variable      | Emission control rate                                                           |
| $\alpha$        | `elast_mrg_cons_utility`          | 1.45                   | Elasticity of marginal utility of consumption                                   |
| $\gamma$        | `elast_capt_in_prod`              | 0.300                  | Elasticity of capital in production function                                    |
| $\rho$          | `social_time_pref_rate`           | 0.015                  | Initial rate of social time preference per year                                 |
|                 | `depr_Capt`                       | 0.1                    | Depreciation rate on capital (per year)                                         |
| $K$             | `Capt0`                           | 223.3                  | Initial world capital 2015 (trillion 2010 USD)                                  |
| $F$             | `ncrb_RFor0`                      | 0.5                    | 2015 forcings of non-CO2 GHG (Wm-2)                                             |
|                 | `ncrb_RFor1`                      | 1.0                    | 2100 forcings of non-CO2 GHG (Wm-2)                                             |
|                 | `t_ncrb_RFor`                     | 18                     | Period when non-CO2 forcings switch                                             |
|                 | `incr_RFor_dbl_crbn`              | 3.6813                 | Forcing from doubling CO2 (Wm-2) 2015                                           |
|                 | `incr_temp_dbl_crb`               | 3.1                    | Equilibrium temperature impact (°C per doubling CO2)                            |
| $\pi_1$         | `a1`                              | 0                      | Damage function intercept                                                       |
| $\pi_2$         | `a2`                              | 0.00236                | Damage function quadratic term                                                  |
|                 | `a3`                              | 2.00                   | Damage function exponent                                                        |
| $\theta_2$      | `cost2_Abat`                      | 2.6                    | Exponent of control cost function                                               |
|                 | `decl_Bstp`                       | 0.025                  | Initial cost decline backstop cost per period                                   |
|                 | `Bstp0`                           | 550                    | Cost of backstop technology 2010 USD per ton CO2                                |
| $\mu_0$         | `ECtr0`                           | 0.03                   | Initial emissions control rate 2015                                             |
|                 | `ECtr_lim`                        | 1.2                    | Upper limit on control rate after 2150                                          |
| $M_{AT}$        | `CStk_atmo0`                      | 851                    | Initial carbon stock in atmosphere 2015 (GtC)                                   |
| $M_{AT}(1750)$  | `CStk_atmo1750`                   | 588                    | Initial carbon stock in 1750                                                    |
| $M_{UP}$        | `CStk_ocup0`                      | 460                    | Initial carbon stock in upper strata 2015 (GtC)                                 |
| $M_{LO}$        | `CStk_oclo0`                      | 1740                   | Initial carbon stock in lower strata 2015 (GtC)                                 |
|                 | `CStk_atmo_eq`                    | 588                    | Equilibrium carbon stock atmosphere (GtC)                                       |
|                 | `CStk_ocup_eq`                    | 360                    | Equilibrium carbon stock upper strata (GtC)                                     |
|                 | `CStk_oclo_eq`                    | 1720                   | Equilibrium carbon stock lower strata (GtC)                                     |
| $\phi_{12}$     | `b12`                             | 0.12                   | Carbon cycle transition matrix                                                  |
| $\phi_{23}$     | `b23`                             | 0.007                  | Carbon cycle transition matrix                                                  |
| $T_{AT}$        | `Temp_atmo0`                      | 0.85                   | Initial atmospheric temperature change (°C from 1900)                           |
| $T_{LO}$        | `Temp_ocea0`                      | 0.0068                 | Initial lower ocean temperature change (°C from 1900)                           |
| $\xi_1$         | `c1`                              | 0.1005                 | Transfer coefficient climate equation atmoshpere temperature                    |
| $\xi_3$         | `c3`                              | 0.088                  | Transfer coefficient atmo-ocen difference                                       |
| $\xi_4$         | `c4`                              | 0.025                  | Transfer coefficient for lower ocean level                                      |

In [2]:
# Packages used in the notebook
import os as os
import numpy as np
import pandas as pd
import glob

import plotly.graph_objects as go
import plotly.express as px

from plotly.subplots import make_subplots

## Tasks

In the last excercise session we looked at how consumption based utility of the *Economic Part* in DICE determines which state of the world is most optimal. The (gross) production of this first group of state variables can be transferd to the *Emission Part* of the model causing emissions which will affect the second group of state variables. Now both model parts are combined into the same cell, building the almos complete DICE model.

For the next exercise session, I ask you to spend some time on two tasks which are listed below and carry the usual `<..>` indicator where you might have to complete some code. Tasks marked as "(optional)" can be skipped if you are short on time. 

### Combining Economic and Emission Part of DICE Model

- The consumption based utility in our model should be derived from production *after* taking climate damages and abatement cost into consideration, not just by considering gross production. For this do the following steps
    1. Create a new state variable (a new empty array) `Prod_net` and define it as production (`Prod_gross`) after considering fractional damages `Prod_net` = `Prod_gross` (1-`Damg_frac`). 
    2. Create a new state variable (a new empty array) `Prod` and define it as production after damages (`Prod_net`) and after considering abatement cost for avoiding emissions `Prod` = `Prod_net` (1-`Abat_cost`). 
    3. Adjust consumption (`Cons`) to be now dependent on `Prod` considering all utility affecting cost and not just `Prod_gross`.

    4. (**Optional**) For completness, add another state variable indicating the real interest rate `RI`based on pcapita consumption `RI` = 1 + `social_time_pref_rate` * `Cons_pcap` / `Cons_pcap`t-1 ** (`elast_mrg_cons_utility`/`tstep`) - 1
    5. (**Optional**) For completness, add three more state variable indicating the cumlative carbon emissions over time for land use (`CEms_land_cum`), industry use (`CEms_land_cum`) and total emissions (`CEms_tot_cum`)

    6. So far, we only considered the effect of carbon emissions on radiative forcing (increase in temperature). But non-carbon green house gases (GHG) also cause forcing and warming to increase captured in `ncrb_RFor`. Extend the endogenous variable radiative forcing (`RFor`) currently soley dependent on the atmospheric carbon stock by adding forcing increase of non-carbon GHG `ncrb_RFor` (Hint: it is literally just a linear addition of `ncrb_RFor`). 

    7. So far, we considered the price of the backstop technology `Bstp` as constant over time. Adjust the variable to be declining over time as `Bstp` = `Bstp0` * (1-`decl_Bstp`)**(t-1)

    8. Dependent on how many additional variables you were able to define, "switch them on" (remove the comment marks) in the exporting section of the code to add said variables in the plot and results data frame. 

    9. (**Optional**) As mentioned in the las exercise session, for utility to eventually be used by an objective function, it needs to be transformed / summed up from an array as in `Util_cons_disc` into a singular value. For this, define a function for the variable `UTILITY` indicating the sum of the discounted consumption utility array `UTILITY`= `tstep`*`scale1`*sum(`Util_cons_disc`) + `scale2`. Note: these scaling coefficients are derrived from extensive model callibration and carry no particular meaning or interpretation.    

In [4]:
# PARAMETERS --------------------------------------------------------------------------------
T_end   = 100
tstep   = 5
t = np.arange(1, T_end + 1)
NT = len(t)


# elasticities
elast_mrg_cons_utility = 1.45
elast_capt_in_prod     = 0.300

# discounting
social_time_pref_rate  = 0.015
social_time_pref_array = 1/((1+social_time_pref_rate)**(tstep*(t-1))) 



# EXOGENOUS VARS -------------------------------------------------------------------------------------------
# labour (Labr) 
Labr0 = 7403    # inital world population (millions) 2015
Labr = np.full(NT, Labr0)

# total factor productivity (TFP)
TFPr0 = 5.115   # initial level of total factor productivity 2015
TFPr = np.full(NT, TFPr0)  

# decarbonization rate (Dcrb)
Dcrb0 = 0.35032
Dcrb = np.full(NT, Dcrb0)  

# carbon land emissions (CEms_land)
CEms_land0 = 2.6   # initial carbon emissions from land use 2015, (2.6 GtCO2/yr)
CEms_land = np.full(NT, CEms_land0)   

# abatement cost (cost1_Abat)
Bstp0 = 550  # Cost of backstop technology 2010$ per ton CO2
cost1_Abat = np.full(NT, Bstp0)  



# ENDOGENOUS VARS -------------------------------------------------------------------------------------------

# Economic Part ------------------------------
# capital
Capt0           = 223.3
depr_Capt       = 0.1    
Capt            = np.zeros(NT)
Capt[0]         = Capt0
def fCapt(iCapt, iInvs, idx):
    if (idx == 0):
        return Capt0
    else:
        return (1-depr_Capt)**tstep * iCapt[idx-1] + tstep * iInvs[idx-1]  
    

# production
Prod_gross = np.zeros(NT)
def fProd_gross(iTFPr, iLabr, iCapt, idx):
    return iTFPr[idx] * ((iLabr[idx]/1000) ** (1 - elast_capt_in_prod)) * (iCapt[idx] ** elast_capt_in_prod)

Prod_net = np.zeros(NT)
# <.1.>


Prod = np.zeros(NT)
# <.2.>





# consumption 
Cons = np.zeros(NT)
def fCons(iProd_gros, iInvs, idx):  # <.3.>
    return iProd_gros[idx] - iInvs[idx]

Cons_pcap = np.zeros(NT)
def fCons_pcap(iCons, iLabr, idx):
    return 1000 * iCons[idx] / iLabr[idx]


# investment + real interest rate
Invs = np.zeros(NT)
def fInvs(iSavi, iProd_gross, idx):
    return iSavi[idx] * iProd_gross[idx] 

RI = np.zeros(NT)      
def fRI(iCons_pcap, idx):
    if (idx == 0):
        return 0
    # else: 
        # return    # <.4.>



# utility
Util_pcap_cons = np.zeros(NT)
def fUtil_pcap_cons(iCons, iLabr, idx):
    return ((iCons[idx] * 1000 / iLabr[idx]) ** (1 - elast_mrg_cons_utility) -1) / (1 - elast_mrg_cons_utility) - 1  

Util_cons_disc = np.zeros(NT)
def fUtil_cons_disc(iUtil_pcap_cons, iLabr, idx):
    return iUtil_pcap_cons[idx] * iLabr[idx] * social_time_pref_array[idx]  


scale1 = 0.0302455265681763,      # Multiplicative scaling coefficient  
scale2 = -10993.704,              # Additive scaling coefficient 
# def fUTILITY(iUtil_cons_disc, resUtility):    #<.9.>
    



# Emissions Part ------------------------------

# carbon emissions (industry and total)
CEms_indu = np.zeros(NT)
def fCEms_indu(iProd_gross, iECtr, iDcrb, idx):
    return iDcrb[idx] * iProd_gross[idx] * (1 - iECtr[idx])

CEms = np.zeros(NT)
def fCEms(iCEms_indu, iCEms_land, idx):
    return iCEms_indu[idx] + iCEms_land[idx]

CEms_indu_cum = np.zeros(NT)
# <.5.>


CEms_land_cum = np.zeros(NT)
# <.5.>

CEms_tot_cum = np.zeros(NT)
# <.5.>


# radiative forcing CO2
incr_RFor_dbl_crbn = 3.6813                   # Forcing from doubling CO2 (Wm-2) 2015
incr_temp_dbl_crb = 3.1                      # Equilibrium temp impact (oC per doubling CO2)
CStk_atmo0    =  851                      # Initial Concentration in atmosphere 2015 (GtC)
CStk_atmo1750 =  588                      # Initial Concentration in 1750, 

# radiative forcing non CO2
ncrb_RFor0 = 0.5                      # 2015 forcings of non-CO2 GHG (Wm-2)
ncrb_RFor1 = 1.0                      # 2100 forcings of non-CO2 GHG (Wm-2)
t_ncrb_RFor = 18                       # Period when non-CO2 forcings switch

ncrb_RFor = np.full(NT, ncrb_RFor0)
ncrb_RFor[0:t_ncrb_RFor] = ncrb_RFor[0:t_ncrb_RFor] + (1/(t_ncrb_RFor-1))*(ncrb_RFor1-ncrb_RFor0)*(t[0:t_ncrb_RFor]-1)
ncrb_RFor[t_ncrb_RFor:NT] = ncrb_RFor[t_ncrb_RFor:NT] + (ncrb_RFor1-ncrb_RFor0)

RFor = np.zeros(NT)
def fRFor(iCStk_atmo, idx):
    return incr_RFor_dbl_crbn * np.log(iCStk_atmo[idx] / CStk_atmo1750) / np.log(2)   # <.6.> 


# damages
a1 = 0                       # Damage function intercept
a2 = 0.00236               # Damage function quadratic term
a3 = 2.00                  # Damage function exponent

Damg_frac = np.zeros(NT)
def fDamg_frac(iTemp_atmo, idx):
    return a1 * iTemp_atmo[idx] + a2*iTemp_atmo[idx]**a3  

Damg = np.zeros(NT)
def fDamg(iProd_gross, iDamg_frac, idx):
    return iProd_gross[idx] * iDamg_frac[idx]


# abatement cost
cost2_Abat = 2.6         

decl_Bstp = 0.025                    # Initial cost decline backstop  cost per period
Bstp0 = 550                      # Cost of backstop technology 2010$ per ton CO2

Bstp = np.full(NT, Bstp0)          # old constant backstop price
# Bstp = # <.7.>  # backstop price


Abat_cost = np.zeros(NT)
def fAbat_cost(iProd_gross, iECtr, icost1_Abat, idx):
    return iProd_gross[idx] * icost1_Abat[idx] * iECtr[idx] ** cost2_Abat

Abat_MC = np.zeros(NT)
def fAbat_MC(iECtr, idx):
    return Bstp[idx] * iECtr[idx] ** (cost2_Abat-1)


# carbon cycle ocean transition matrix
CStk_ocup0    =  460                      # Initial Concentration in upper strata 2015 (GtC)
CStk_oclo0    =  1740                     # Initial Concentration in lower strata 2015 (GtC)
CStk_atmo_eq  =  588                      # Equilibrium concentration atmosphere (GtC)
CStk_ocup_eq  =  360                      # Equilibrium concentration upper strata (GtC)
CStk_oclo_eq  =  1720                     # Equilibrium concentration lower strata (GtC)
b12           =  0.12                     # carbon cycle transition matrix (atmo to up)
b23           =  0.007                    # carbon cycle transition matrix (up to lo)

b11 = 1 - b12 
b21 = b12*(CStk_ocup_eq/CStk_atmo_eq)   
b22 = 1 - b21 - b23
b32 = b23*(CStk_oclo_eq/CStk_ocup_eq) 
b33 = 1 - b32

CStk_atmo       = np.zeros(NT)
CStk_atmo[0]    = CStk_atmo0
def fCStk_atmo(iCStk_atmo, iECtr, iCEms, idx):
    if (idx == 0):
        return CStk_atmo0
    else:
        return iCStk_atmo[idx-1]*b11 + CStk_ocup[idx-1]*b21 + iCEms[idx-1] * tstep/3.666    

CStk_ocup       = np.zeros(NT)
CStk_ocup[0]    = CStk_ocup0
def fCStk_ocup(iCStk_atmo, iCStk_ocup, iCStk_oclo, idx):
    if (idx == 0):
        return CStk_ocup0
    else:
        return iCStk_atmo[idx-1]*b12 + iCStk_ocup[idx-1]*b22 + iCStk_oclo[idx-1]*b32  

CStk_oclo       = np.zeros(NT)
CStk_oclo[0]    = CStk_oclo0
def fCStk_oclo(iCStk_ocup, iCStk_oclo, idx):
    if (idx == 0):
        return CStk_oclo0
    else:
        return iCStk_oclo[idx-1]*b33 + iCStk_ocup[idx-1]*b23
    

# temperature
Temp_atmo0 = 0.85                    # Initial atmospheric temp change (C from 1900)
Temp_ocea0 = 0.0068                   # Initial lower stratum temp change (C from 1900)
c1         = 0.1005                   # Climate equation coefficient for upper level
c3         = 0.088                    # Transfer coefficient upper to lower stratum
c4         = 0.025                    # Transfer coefficient for lower level

Temp_atmo       = np.zeros(NT)
Temp_atmo[0]    = Temp_atmo0
def fTemp_atmo(iTemp_atmo, iRFor, iTemp_ocea, idx):
    if (idx == 0):
        return Temp_atmo0
    else:
        return iTemp_atmo[idx-1] + c1*(iRFor[idx] - (incr_RFor_dbl_crbn/incr_temp_dbl_crb)*iTemp_atmo[idx-1] - c3*(iTemp_atmo[idx-1] - iTemp_ocea[idx-1])) 

Temp_ocea       = np.zeros(NT)
Temp_ocea[0]    = Temp_ocea0
def fTemp_ocea(iTemp_atmo, iTemp_ocea, idx):
    if (idx == 0):
        return Temp_ocea0
    else:
        return iTemp_ocea[idx-1] + c4*(iTemp_atmo[idx-1] - iTemp_ocea[idx-1])



# POSSIBLE HUMAN ACTION ------------------------------------------------------------ 
Savi = np.full(NT, 0.1)
ECtr = np.full(NT, 0.1) 



# UPDATE STATE VARIABLES over time ------------------------------------------------------------ 
for i in range(NT):
    Capt[i]            = fCapt(Capt, Invs, i)                         # <.8.>
    Prod_gross[i]      = fProd_gross(TFPr, Labr, Capt, i)
    # Prod_net[i]        = fProd_net(Prod_gross, Damg_frac, i)
    # Prod[i]            = fProd(Prod_net, Abat_cost, i)
    Invs[i]            = fInvs(Savi, Prod, i)
    Cons[i]            = fCons(Prod_gross, Invs, i)    # <.3.>
    Cons_pcap[i]       = fCons_pcap(Cons, Labr, i)
    # RI[i]              = fRI(Cons_pcap, i)
    Util_pcap_cons[i]  = fUtil_pcap_cons(Cons, Labr, i)
    Util_cons_disc[i]  = fUtil_cons_disc(Util_pcap_cons, Labr, i)

    CEms_indu[i]       = fCEms_indu(Prod_gross, ECtr, Dcrb, i)
    CEms[i]            = fCEms(CEms_indu, CEms_land, i)            
    # CEms_indu_cum[i]   = fCEms_indu_cum(CEms_indu_cum, CEms_indu, i)
    # CEms_land_cum[i]   = fCEms_land_cum(CEms_land_cum, CEms_land, i)
    # CEms_tot_cum[i]    = fCEms_tot_cum(CEms_indu_cum, CEms_land_cum, i)
    CStk_atmo[i]       = fCStk_atmo(CStk_atmo, ECtr, CEms, i)
    CStk_oclo[i]       = fCStk_oclo(CStk_ocup, CStk_oclo, i)
    CStk_ocup[i]       = fCStk_ocup(CStk_atmo, CStk_ocup, CStk_oclo, i)
    RFor[i]            = fRFor(CStk_atmo, i)
    Temp_atmo[i]       = fTemp_atmo(Temp_atmo, RFor, Temp_ocea, i)
    Temp_ocea[i]       = fTemp_ocea(Temp_atmo, Temp_ocea, i)
    Damg_frac[i]       = fDamg_frac(Temp_atmo, i)
    Damg[i]            = fDamg(Prod_gross, Damg_frac, i)
    Abat_cost[i]       = fAbat_cost(Prod_gross, ECtr, cost1_Abat, i)
    Abat_MC[i]         = fAbat_MC(ECtr, i)




# EXPORT RESULTS ------------------------------------------------------------
tuples_to_plot = (
    (Capt,            'Capt'),         # <.8.>
    (Prod_gross,      'Prod_gross'), 
    (Prod_net,        'Prod_net'), 
    (Prod,            'Prod'), 
    (Invs,            'Invs'), 
    (Cons,            'Cons'), 
    (Cons_pcap,       'Cons_pcap'), 
    (RI,              'RI'), 
    (Util_pcap_cons,  'Util_pcap_cons'), 
    (Util_cons_disc,  'Util_cons_disc'), 

    (CEms_indu,       'CEms_indu'), 
    (CEms,            'CEms'), 
    (CEms_indu_cum,   'CEms_indu_cum'), 
    (CEms_land_cum,   'CEms_land_cum'), 
    (CEms_tot_cum,    'CEms_tot_cum'), 
    (CStk_atmo,       'CStk_atmo'), 
    (CStk_oclo,       'CStk_oclo'), 
    (CStk_ocup,       'CStk_ocup'), 
    (RFor,            'RFor'), 
    (Temp_atmo,       'Temp_atmo'), 
    (Temp_ocea,       'Temp_ocea'), 
    (Damg_frac,       'Damg_frac'), 
    (Damg,            'Damg'), 
    (Abat_cost,       'Abat_cost'), 
    (Abat_MC,         'Abat_MC'), 
)
results_df = pd.DataFrame({'t': t, 't_year': 2000 + t*tstep})
for var_object, col_name in tuples_to_plot:
    results_df[col_name] = var_object



# # VISUALIZATION ------------------------------------------------------------
cols_to_plot = [col for col in results_df.columns if col not in ['t', 't_year']]
n_rows = len(cols_to_plot) // 3 + 1 if len(cols_to_plot) % 3 > 0 else len(cols_to_plot) // 3
fig = make_subplots(rows=n_rows, cols =3, subplot_titles=cols_to_plot)
for i, col in enumerate(cols_to_plot):
    row_idx = i // 3 + 1
    col_idx = i % 3 + 1
    fig.add_trace(go.Scatter(x=results_df['t_year'], y=results_df[col], name=col), row=row_idx, col=col_idx)
    # fig.update_xaxes(title_text="Year", row=row_idx, col=col_idx)

fig.update_layout(title_text="Economic State variables", template = 'plotly_white', )
fig.show()


## Additinal Steps in Class

After having fully completed the DICE model with all its components, it is time to relax another simplifying assumption we have made so far: constant exogenous variables. As mentioned in class, exogenous variables are assumption based and are there for externally given. For coding simplicity, so far we have always assumed these variables to be constant (to the inital value) over time. Now the exogenous variables depend on a data frame I have given you, containing the default values also used in the DICE model. 

Try to define your exogenous variables using the data frame `DICE_exogn_vars_dynamic.csv`. To do that, you have to
1. Allow Google Colab to access your Google Drive
2. Import the CSV data frame. 
3. Assign the values from the columns in the data frame to your variable arrays
4. Export your results as a csv to a folder called "output"
5. Compare two model runs without any optimization, just different exogenous variables (constant vs dynamic based on `DICE_exogn_vars_dynamic.csv`)

### (Pre-requisits)
Besides the ususal packages, following cells contain code to make the notebook run as intended on Google Colab (so you can write / read documents directly from / to your drive) and a visualization function that is needed in class to visualize model results quickly. 

In [ ]:
# <.1.>  

# Cell should only apply when running this notebook on google COLAB
# > asks for access to Drive and permission to read and write files
path_where_notebook_is_stored_on_GDrive = 'MyDrive/...'
import os as os


# set working directory to the location of this file, either in Google Colab
if 'content' in os.getcwd():
  from google.colab import drive
  drive.mount('/content/drive')
  os.chdir(f'/content/drive/{path_where_notebook_is_stored_on_GDrive}')


In [5]:
# scen_name = 'DICE_exogn_constant'
scen_name = 'DICE_exogn_dynamic'

# PARAMETERS --------------------------------------------------------------------------------
T_end   = 100
tstep   = 5
t = np.arange(1, T_end + 1)
NT = len(t)


# elasticities
elast_mrg_cons_utility = 1.45
elast_capt_in_prod     = 0.300

# discounting
social_time_pref_rate  = 0.015
social_time_pref_array = 1/((1+social_time_pref_rate)**(tstep*(t-1))) 



# EXOGENOUS VARS -------------------------------------------------------------------------------------------

# old exogneous vars
Labr0 = 7403    # inital world population (millions) 2015
Labr = np.full(NT, Labr0)
TFPr0 = 5.115   # initial level of total factor productivity 2015
TFPr = np.full(NT, TFPr0)  
Dcrb0 = 0.35032
Dcrb = np.full(NT, Dcrb0)  
CEms_land0 = 2.6   # initial carbon emissions from land use 2015, (2.6 GtCO2/yr)
CEms_land = np.full(NT, CEms_land0)   
Bstp0 = 550  # Cost of backstop technology 2010$ per ton CO2
cost1_Abat = np.full(NT, Bstp0) 


# exog_Vars_DICE =  # <.2.>  # read exogenous variables from input file
# Labr = exog_Vars_DICE['Labr'].values
# TFPr = # <.3.>



# ENDOGENOUS VARS -------------------------------------------------------------------------------------------

# Economic Part ------------------------------
# capital
Capt0           = 223.3
depr_Capt       = 0.1    
Capt            = np.zeros(NT)
Capt[0]         = Capt0
def fCapt(iCapt, iInvs, idx):
    if (idx == 0):
        return Capt0
    else:
        return (1-depr_Capt)**tstep * iCapt[idx-1] + tstep * iInvs[idx-1]  
    

# production
Prod_gross = np.zeros(NT)
def fProd_gross(iTFPr, iLabr, iCapt, idx):
    return iTFPr[idx] * ((iLabr[idx]/1000) ** (1 - elast_capt_in_prod)) * (iCapt[idx] ** elast_capt_in_prod)

Prod_net = np.zeros(NT)
def fProd_net(iProd_gross, iDamg_frac, idx):   
    return iProd_gross[idx]    *  (1 - iDamg_frac[idx])  

Prod = np.zeros(NT)
def fProd(iProd_net, iAbat_cost, idx):          
    return iProd_net[idx]  -  iAbat_cost[idx]




# consumption 
Cons = np.zeros(NT)
def fCons(iProd, iInvs, idx): 
    return iProd[idx] - iInvs[idx]

Cons_pcap = np.zeros(NT)
def fCons_pcap(iCons, iLabr, idx):
    return 1000 * iCons[idx] / iLabr[idx]


# investment + real interest rate
Invs = np.zeros(NT)
def fInvs(iSavi, iProd_gross, idx):
    return iSavi[idx] * iProd_gross[idx] 

RI = np.zeros(NT)       
def fRI(iCons_pcap, idx):
    if (idx == 0):
        return 0
    else: 
        return (1 + social_time_pref_rate) * (iCons_pcap[idx] / iCons_pcap[idx-1]) ** (elast_mrg_cons_utility/tstep) - 1



# utility
Util_pcap_cons = np.zeros(NT)
def fUtil_pcap_cons(iCons, iLabr, idx):
    return ((iCons[idx] * 1000 / iLabr[idx]) ** (1 - elast_mrg_cons_utility) -1) / (1 - elast_mrg_cons_utility) - 1  

Util_cons_disc = np.zeros(NT)
def fUtil_cons_disc(iUtil_pcap_cons, iLabr, idx):
    return iUtil_pcap_cons[idx] * iLabr[idx] * social_time_pref_array[idx]  


scale1 = 0.0302455265681763,      # Multiplicative scaling coefficient  
scale2 = -10993.704,              # Additive scaling coefficient 
def fUTILITY(iUtil_cons_disc, resUtility):   
    resUtility[0] = tstep * scale1 * np.sum(iUtil_cons_disc) + scale2
    



# Emissions Part ------------------------------

# carbon emissions (industry and total)
CEms_indu = np.zeros(NT)
def fCEms_indu(iProd_gross, iECtr, iDcrb, idx):
    return iDcrb[idx] * iProd_gross[idx] * (1 - iECtr[idx])

CEms = np.zeros(NT)
def fCEms(iCEms_indu, iCEms_land, idx):
    return iCEms_indu[idx] + iCEms_land[idx]

CEms_indu_cum = np.zeros(NT)
def fCEms_indu_cum(iCEms_indu_cum, iCEms_indu, idx):           
    return iCEms_indu_cum[idx-1] + iCEms_indu[idx-1] * (tstep / 3.666)

CEms_land_cum = np.zeros(NT)
def fCEms_land_cum(iCEms_land_cum, iCEms_land, idx):           
    return iCEms_land_cum[idx-1] + iCEms_land[idx-1] * (tstep / 3.666)
CEms_tot_cum = np.zeros(NT)
                
def fCEms_tot_cum(iCEms_indu_cum, iCEms_land_cum, idx):        
    return iCEms_indu_cum[idx] + iCEms_land_cum[idx] 



# radiative forcing CO2
incr_RFor_dbl_crbn = 3.6813                   # Forcing from doubling CO2 (Wm-2) 2015
incr_temp_dbl_crb = 3.1                      # Equilibrium temp impact (oC per doubling CO2)
CStk_atmo0    =  851                      # Initial Concentration in atmosphere 2015 (GtC)
CStk_atmo1750 =  588                      # Initial Concentration in 1750, 

# radiative forcing non CO2
ncrb_RFor0 = 0.5                      # 2015 forcings of non-CO2 GHG (Wm-2)
ncrb_RFor1 = 1.0                      # 2100 forcings of non-CO2 GHG (Wm-2)
t_ncrb_RFor = 18                       # Period when non-CO2 forcings switch

ncrb_RFor = np.full(NT, ncrb_RFor0)
ncrb_RFor[0:t_ncrb_RFor] = ncrb_RFor[0:t_ncrb_RFor] + (1/(t_ncrb_RFor-1))*(ncrb_RFor1-ncrb_RFor0)*(t[0:t_ncrb_RFor]-1)
ncrb_RFor[t_ncrb_RFor:NT] = ncrb_RFor[t_ncrb_RFor:NT] + (ncrb_RFor1-ncrb_RFor0)

RFor = np.zeros(NT)
def fRFor(iCStk_atmo, idx):
    return incr_RFor_dbl_crbn * np.log(iCStk_atmo[idx] / CStk_atmo1750) / np.log(2)   + ncrb_RFor[idx]  # <.6.> 


# damages
a1 = 0                       # Damage function intercept
a2 = 0.00236               # Damage function quadratic term
a3 = 2.00                  # Damage function exponent

Damg_frac = np.zeros(NT)
def fDamg_frac(iTemp_atmo, idx):
    return a1 * iTemp_atmo[idx] + a2*iTemp_atmo[idx]**a3  

Damg = np.zeros(NT)
def fDamg(iProd_gross, iDamg_frac, idx):
    return iProd_gross[idx] * iDamg_frac[idx]


# abatement cost
cost2_Abat = 2.6         

decl_Bstp = 0.025                    # Initial cost decline backstop  cost per period
Bstp0 = 550                      # Cost of backstop technology 2010$ per ton CO2

Bstp = Bstp0 * (1-decl_Bstp)**(t-1)    # backstop price


Abat_cost = np.zeros(NT)
def fAbat_cost(iProd_gross, iECtr, icost1_Abat, idx):
    return iProd_gross[idx] * icost1_Abat[idx] * iECtr[idx] ** cost2_Abat

Abat_MC = np.zeros(NT)
def fAbat_MC(iECtr, idx):
    return Bstp[idx] * iECtr[idx] ** (cost2_Abat-1)


# carbon cycle ocean transition matrix
CStk_ocup0    =  460                      # Initial Concentration in upper strata 2015 (GtC)
CStk_oclo0    =  1740                     # Initial Concentration in lower strata 2015 (GtC)
CStk_atmo_eq  =  588                      # Equilibrium concentration atmosphere (GtC)
CStk_ocup_eq  =  360                      # Equilibrium concentration upper strata (GtC)
CStk_oclo_eq  =  1720                     # Equilibrium concentration lower strata (GtC)
b12           =  0.12                     # carbon cycle transition matrix (atmo to up)
b23           =  0.007                    # carbon cycle transition matrix (up to lo)

b11 = 1 - b12 
b21 = b12*(CStk_ocup_eq/CStk_atmo_eq)   
b22 = 1 - b21 - b23
b32 = b23*(CStk_oclo_eq/CStk_ocup_eq) 
b33 = 1 - b32

CStk_atmo       = np.zeros(NT)
CStk_atmo[0]    = CStk_atmo0
def fCStk_atmo(iCStk_atmo, iECtr, iCEms, idx):
    if (idx == 0):
        return CStk_atmo0
    else:
        return iCStk_atmo[idx-1]*b11 + CStk_ocup[idx-1]*b21 + iCEms[idx-1] * tstep/3.666    

CStk_ocup       = np.zeros(NT)
CStk_ocup[0]    = CStk_ocup0
def fCStk_ocup(iCStk_atmo, iCStk_ocup, iCStk_oclo, idx):
    if (idx == 0):
        return CStk_ocup0
    else:
        return iCStk_atmo[idx-1]*b12 + iCStk_ocup[idx-1]*b22 + iCStk_oclo[idx-1]*b32  

CStk_oclo       = np.zeros(NT)
CStk_oclo[0]    = CStk_oclo0
def fCStk_oclo(iCStk_ocup, iCStk_oclo, idx):
    if (idx == 0):
        return CStk_oclo0
    else:
        return iCStk_oclo[idx-1]*b33 + iCStk_ocup[idx-1]*b23
    

# temperature
Temp_atmo0 = 0.85                    # Initial atmospheric temp change (C from 1900)
Temp_ocea0 = 0.0068                   # Initial lower stratum temp change (C from 1900)
c1         = 0.1005                   # Climate equation coefficient for upper level
c3         = 0.088                    # Transfer coefficient upper to lower stratum
c4         = 0.025                    # Transfer coefficient for lower level

Temp_atmo       = np.zeros(NT)
Temp_atmo[0]    = Temp_atmo0
def fTemp_atmo(iTemp_atmo, iRFor, iTemp_ocea, idx):
    if (idx == 0):
        return Temp_atmo0
    else:
        return iTemp_atmo[idx-1] + c1*(iRFor[idx] - (incr_RFor_dbl_crbn/incr_temp_dbl_crb)*iTemp_atmo[idx-1] - c3*(iTemp_atmo[idx-1] - iTemp_ocea[idx-1])) 

Temp_ocea       = np.zeros(NT)
Temp_ocea[0]    = Temp_ocea0
def fTemp_ocea(iTemp_atmo, iTemp_ocea, idx):
    if (idx == 0):
        return Temp_ocea0
    else:
        return iTemp_ocea[idx-1] + c4*(iTemp_atmo[idx-1] - iTemp_ocea[idx-1])



# POSSIBLE HUMAN ACTION ------------------------------------------------------------ 
Savi = np.full(NT, 0.1)
ECtr = np.full(NT, 0.1) 



# UPDATE STATE VARIABLES over time ------------------------------------------------------------ 
for i in range(NT):
    Capt[i]            = fCapt(Capt, Invs, i)                       
    Prod_gross[i]      = fProd_gross(TFPr, Labr, Capt, i)
    Prod_net[i]        = fProd_net(Prod_gross, Damg_frac, i)
    Prod[i]            = fProd(Prod_net, Abat_cost, i)
    Invs[i]            = fInvs(Savi, Prod, i)
    Cons[i]            = fCons(Prod, Invs, i)    
    Cons_pcap[i]       = fCons_pcap(Cons, Labr, i)
    RI[i]              = fRI(Cons_pcap, i)
    Util_pcap_cons[i]  = fUtil_pcap_cons(Cons, Labr, i)
    Util_cons_disc[i]  = fUtil_cons_disc(Util_pcap_cons, Labr, i)

    CEms_indu[i]       = fCEms_indu(Prod_gross, ECtr, Dcrb, i)
    CEms[i]            = fCEms(CEms_indu, CEms_land, i)            
    CEms_indu_cum[i]   = fCEms_indu_cum(CEms_indu_cum, CEms_indu, i)
    CEms_land_cum[i]   = fCEms_land_cum(CEms_land_cum, CEms_land, i)
    CEms_tot_cum[i]    = fCEms_tot_cum(CEms_indu_cum, CEms_land_cum, i)
    CStk_atmo[i]       = fCStk_atmo(CStk_atmo, ECtr, CEms, i)
    CStk_oclo[i]       = fCStk_oclo(CStk_ocup, CStk_oclo, i)
    CStk_ocup[i]       = fCStk_ocup(CStk_atmo, CStk_ocup, CStk_oclo, i)
    RFor[i]            = fRFor(CStk_atmo, i)
    Temp_atmo[i]       = fTemp_atmo(Temp_atmo, RFor, Temp_ocea, i)
    Temp_ocea[i]       = fTemp_ocea(Temp_atmo, Temp_ocea, i)
    Damg_frac[i]       = fDamg_frac(Temp_atmo, i)
    Damg[i]            = fDamg(Prod_gross, Damg_frac, i)
    Abat_cost[i]       = fAbat_cost(Prod_gross, ECtr, cost1_Abat, i)
    Abat_MC[i]         = fAbat_MC(ECtr, i)




# EXPORT RESULTS ------------------------------------------------------------
tuples_to_plot = (
    (Capt,            'Capt'),       
    (Prod_gross,      'Prod_gross'), 
    (Prod_net,        'Prod_net'), 
    (Prod,            'Prod'), 
    (Invs,            'Invs'), 
    (Cons,            'Cons'), 
    (Cons_pcap,       'Cons_pcap'), 
    (RI,              'RI'), 
    (Util_pcap_cons,  'Util_pcap_cons'), 
    (Util_cons_disc,  'Util_cons_disc'), 

    (CEms_indu,       'CEms_indu'), 
    (CEms,            'CEms'), 
    (CEms_indu_cum,   'CEms_indu_cum'), 
    (CEms_land_cum,   'CEms_land_cum'), 
    (CEms_tot_cum,    'CEms_tot_cum'), 
    (CStk_atmo,       'CStk_atmo'), 
    (CStk_oclo,       'CStk_oclo'), 
    (CStk_ocup,       'CStk_ocup'), 
    (RFor,            'RFor'), 
    (Temp_atmo,       'Temp_atmo'), 
    (Temp_ocea,       'Temp_ocea'), 
    (Damg_frac,       'Damg_frac'), 
    (Damg,            'Damg'), 
    (Abat_cost,       'Abat_cost'), 
    (Abat_MC,         'Abat_MC'), 
)
results_df = pd.DataFrame({'scen_name': scen_name, 't': t, 't_year': 2000 + t*tstep})
for var_object, col_name in tuples_to_plot:
    results_df[col_name] = var_object

os.makedirs('output/', exist_ok=True)
# results_df.to_csv(...)  # <.4.>  # create output directory if it does not exist




# # VISUALIZATION ------------------------------------------------------------
cols_to_plot = [col for col in results_df.columns if col not in ['t', 't_year']]
n_rows = len(cols_to_plot) // 3 + 1 if len(cols_to_plot) % 3 > 0 else len(cols_to_plot) // 3
fig = make_subplots(rows=n_rows, cols =3, subplot_titles=cols_to_plot)
for i, col in enumerate(cols_to_plot):
    row_idx = i // 3 + 1
    col_idx = i % 3 + 1
    fig.add_trace(go.Scatter(x=results_df['t_year'], y=results_df[col], name=col), row=row_idx, col=col_idx)

fig.update_layout(title_text="Economic State variables", template = 'plotly_white', )
fig.show()




In [6]:

def plot_comparison(output_dir_name = 'output', title = 'DICE Economic and Emission variables', n_columns: int = 4, fig_indv_show: bool = False, fig_comp_show: bool = True):
    TT = np.linspace(2000, 2500, 100, dtype=np.int32)
    os.makedirs('plots/', exist_ok=True)

    # get all model results in output directory + cosncatenate to 1 df
    result_df_paths = sorted(glob.glob(f'{output_dir_name}/*results_df.csv'))
    
    mod_comp_list = []
    for path in result_df_paths:
        df = pd.read_csv(path)
        mod_comp_list.append(df)

    mod_comp_df = pd.concat(mod_comp_list, axis=0)


    # model individual plots ------------------------------
    cols_to_plot = [col for col in mod_comp_df.columns if col not in ['scen_name', 't', 't_year']]
    n_rows = len(cols_to_plot) // n_columns + 1 if len(cols_to_plot) % n_columns != 0 else len(cols_to_plot) // n_columns

    for scen in mod_comp_df['scen_name'].unique():

        fig = make_subplots(rows=n_rows, cols=n_columns, subplot_titles=cols_to_plot)
        for i, col_plot in enumerate(cols_to_plot):
            row = i // n_columns + 1
            col = i % n_columns + 1
            fig.add_trace(go.Scatter(x=TT,
                                    y=mod_comp_df.loc[mod_comp_df['scen_name'] == scen, col_plot].values,
                                    mode='lines', name=col_plot), row=row, col=col)

        fig.update_layout(title_text=f"{title} ({scen})", template='plotly_white', showlegend=False)
        fig.write_html(f'plots/{scen}.html')
        fig.show() if fig_indv_show else None


    # model comparison plot ------------------------------
    color_scale = px.colors.sequential.Jet
    fig_comp               = make_subplots(rows=n_rows, cols=n_columns, subplot_titles=cols_to_plot)
    fig_comp_scen_legend   = make_subplots(rows=n_rows, cols=n_columns, subplot_titles=cols_to_plot)
    for i, col_plot in enumerate(cols_to_plot):
        for j, scen in enumerate(mod_comp_df['scen_name'].unique()):
            row = i // n_columns + 1
            col = i % n_columns + 1
            scen_color = color_scale[j % len(color_scale)]

            fig_comp.add_trace(go.Scatter(x=TT,
                                                y=mod_comp_df.loc[mod_comp_df['scen_name'] == scen, col_plot].values,
                                                mode='lines',
                                                name=f'{col_plot}_{scen}',
                                                # legendgroup=f'{scen}',
                                                ), row=row, col=col)
            showlegend_TF = True if i == 0 else False
            fig_comp_scen_legend.add_trace(go.Scatter(x=TT,
                                                y=mod_comp_df.loc[mod_comp_df['scen_name'] == scen, col_plot].values,
                                                mode='lines',
                                                line = dict(color = scen_color),
                                                name=f'traces_{scen}',
                                                legendgroup=f'{scen}',
                                                showlegend = showlegend_TF,
                                                ), row=row, col=col)
        fig_comp.add_trace(go.Scatter(x=[None,],
                                            y=[None,],
                                            mode='lines', name='-', opacity=0), row=row, col=col)


    fig_comp.update_layout(title_text=f"Comparison {title}", template='plotly_white', showlegend=True)
    fig_comp.write_html('plots/DICE_comparison.html')
    fig_comp_scen_legend.update_layout(title_text=f"Comparison {title}", template='plotly_white', showlegend=True)
    fig_comp_scen_legend.write_html('plots/DICE_comparison_simple_legend.html')

    fig_comp.show() if fig_comp_show else None
    fig_comp_scen_legend.show() if fig_comp_show else None


 # <.5.>  
